In [1]:
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from importlib import reload
from shutil import rmtree

%matplotlib inline

# Task 1

In [2]:
from data_preparation import read_images, batch, similarity
from model import cnn_model_fn

## Loading Data

In [3]:
N_CLASSES = len(os.listdir('dataset/coarse/'))

coarse = read_images('dataset/coarse/')
fine = read_images('dataset/fine/')
real = read_images('dataset/real/')
train_idxs = ()
with open('dataset/real/training_split.txt') as f:
    train_idxs = set(int(x) for x in f.read().split(', '))
test_idxs = set(range(len(real))) - train_idxs
Sdb    = coarse
Strain = fine + [real[i] for i in train_idxs]
Stest  = [real[i] for i in test_idxs]

# Task 2

Creating the neural network

### Cache

In [4]:
import os

CACHE_DIR = 'cache/'
#rmtree(CACHE_DIR)
#os.makedirs(CACHE_DIR)
MODEL_PATH = os.path.join(CACHE_DIR,'cnn_model')
INPUT_PATH = os.path.join(CACHE_DIR,'cnn_input')
#os.makedirs(MODEL_PATH)
#os.makedirs(INPUT_PATH)

Define Model

In [5]:
cnn_descriptor = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'cache/cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb33314518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
# train input
def gen_train_input_fn(Sdb,Strain, batch_size):
    #Sdb    = features['Sdb']
    #Strain = features['Strain']
    #dataset = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor(batch(Sdb,Strain,batch_size)))
    assert(batch_size % 3 == 0)
    def inner():
        #b = batch(Sdb,Strain,batch_size)
        #dataset = tf.data.Dataset.from_tensor_slices(tf.convert_to_tensor(b))
        dataset = tf.data.Dataset.from_generator(
            batch(Sdb,Strain),
            tf.float32,
            tf.TensorShape([64, 64, 3])
        )
        return dataset.batch(batch_size)
    return inner

In [7]:
train_input_fn = gen_train_input_fn(Sdb,Strain,300)
for x in range(10,1000,10):
    cnn_descriptor.train(
        input_fn=train_input_fn,
        max_steps=x)

INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Skipping training 

In [8]:
from sklearn.neighbors import KNeighborsClassifier

In [9]:
Sdb_img = np.array([x.img for x in Sdb])
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    np.array(Sdb_img),
    shuffle=False,
)

Sdb_descriptors = list(cnn_descriptor.predict(input_fn=eval_input_fn))

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cache/cnn_model/model.ckpt-990
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [10]:
Stest_img = np.array([x.img for x in Stest])
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    Stest_img,
    shuffle=False,
)

Stest_descriptors = list(cnn_descriptor.predict(input_fn=eval_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cache/cnn_model/model.ckpt-990
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [13]:
neigh = KNeighborsClassifier(n_neighbors=1)
X, y = Sdb_descriptors, [x.cls for x in Sdb]
neigh.fit(X,y)

idxs = neigh.kneighbors(Stest_descriptors)[1]

hist = np.array([0,0,0,0])
N = len(Stest_descriptors)
for i in range(N):
    db = Sdb[idxs[i][0]]
    test = Stest[i]

    if db.cls != test.cls:
        continue

    theta = np.rad2deg(similarity(db.quat, test.quat))
    if theta < 10:
        hist[0] += 1
    if theta < 20:
        hist[1] += 1
    if theta < 40:
        hist[2] += 1
    hist[3] += 1

In [15]:
100 * hist / N

array([ 0.        ,  0.12917512,  1.56855508, 16.82967337])

In [26]:
for i,d in enumerate(Sdb_descriptors):
    print(Sdb[i].cls,np.max(d))

0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 0.1526968
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 0.04669378
0 -0.0
0 -0.0
0 0.0105207525
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 0.040376976
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 0.07209955
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 0.08518978
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 0.028880354
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 0.1770436
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.0
0 -0.

3 0.207554
3 0.178126
3 0.22611445
3 0.20877737
3 0.28186154
3 0.18291561
3 0.19764228
3 0.13749732
3 0.08124745
3 0.3591737
3 0.15593195
3 0.20672588
3 0.1836293
3 0.19851214
3 0.2765779
3 0.13200268
3 0.18823302
3 0.26419115
3 0.22079965
3 0.17464751
3 0.27454245
3 0.20554616
3 0.20001186
3 0.23259811
3 0.13230702
3 0.18133911
3 0.1479576
3 0.123533055
3 0.14811099
3 0.21212693
3 0.1322181
3 0.13920295
3 0.11359023
3 0.15736796
3 0.23218934
3 0.18517256
3 0.16865173
3 0.16218393
3 0.2913058
3 0.1838196
3 0.12912336
3 0.20417914
3 0.18680489
3 0.17268793
3 0.22659034
3 0.14026782
3 0.22651537
3 0.2571268
3 0.109995596
3 0.19648382
3 0.16160336
3 0.17966178
3 0.1690512
3 0.2602818
3 0.22563592
3 0.19278027
3 0.15010452
3 0.14716633
3 0.19668627
3 0.2115961
3 0.3062374
3 0.16068219
3 0.17146112
3 0.17446741
3 0.18519773
3 0.17245978
3 0.14772159
3 0.20848729
3 0.17671598
3 0.22403498
3 0.3138336
3 0.1485486
3 0.20166478
3 0.12629506
3 0.18495941
3 0.17796928
3 0.17411709
3 0.15289554
3 